## PLANNING

This notebook describes the planning.py module, which covers Classical Planning from Chapter 10 & Planning & Acting in Real World Chapter 11.

In [1]:
from planning import *
from utils import expr

#PDDL 
The PDDL (Planning Domain Definition Language)   allows us to express all actions with one action schema,

The `PDDL class` include:

1. `init(self, initial_state, actions, goal_test)`: the constructor creates a knowledge base with initial state, initialises actions and goal_test_func function with goal_test.
2. `goal_test(self)`: initialises goal_test with kb.
3. `act(self, action)`:  performs the action given as argument, along with checks preformed on pre-conditions.

PDDL is a domain definition language. It is used to define the properties of a domain, the predicates which are used and the action definition.  A predicate defines the property of an object which can be true or false, e.g. yellow t-shirt. Yellow is the property and t-shirt is the object. 

#ACTIONS
Actions are described by a set of action schemas that implicitly define the class Actions:
Actions consists of a precondition(positive and negative) and effect(positive and negative). It consists of following 

Each problem described in chapter 10 has:
- an initial state
- a goal
- actions with preconditions and effects (these action are split into _pos (positive conditions) and _neg (negative conditions))
Each problem thus requires a solution which is satisfied by some initial conditions and solved using actions which meet the preconditions and effects.
 

# AIR CARGO PROBLEM from Figure 10.1

To define a problem we have to define a initial (predicates which are true at the beginning of the problem) and a goal state (predicates which are true at
 the end of the problem). 


The following code defines the initial state for the problem

In [2]:
def air_cargo():
    init = [expr('At(C1, SFO)'),     
            expr('At(C2, JFK)'),
            expr('At(P1, SFO)'),
            expr('At(P2, JFK)'),
            expr('Cargo(C1)'),
            expr('Cargo(C2)'),
            expr('Plane(P1)'),
            expr('Plane(P2)'),
            expr('Airport(JFK)'),
            expr('Airport(SFO)')]

`init` defines the initial state of the problem.

**`expr('At(C1, SF0)')`** represents the predicate **Cargo 1 at San Fransisco**

**`expr('At(P1, SF0)')`** represents the predicate **Plane 1 at San Fransisco**    ... and so on.




The following code defines the goal_test() function which tests if the solution achieves goal or not. 'required' states the goal:

In [3]:
def goal_test(kb):
        required = [expr('At(C1 , JFK)'), expr('At(C2 ,SFO)')]
        for q in required:
            if kb.ask(q) is False:
                return False
        return True

For this problem the Goal is   **`(At(C1 , JFK ) ∧ At(C2 , SFO))`** i.e., **Cargo 1 ar JFK and Cargo 2 at San Fransisco** which is written as:

`required = [expr('At(C1 , JFK)'), expr('At(C2 ,SFO)')]`

The function `goal_test(kb)` takes a knowledge base as argument, and for every predicate in `required` (goal), it checks the `ask` function from `KB class`. The `ask` function returns value `True` or `False` accordingly if predicate in `required` meets preconditions or not (defined ahead).

Actions such as 'load', 'unload'  and fly are defined with preconditions and effects accompanying them.

In [4]:
# Actions
#  Load
precond_pos = [expr("At(c, a)"), expr("At(p, a)"), expr("Cargo(c)"), expr("Plane(p)"), expr("Airport(a)")]
precond_neg = []
effect_add = [expr("In(c, p)")]
effect_rem = [expr("At(c, a)")]
load = Action(expr("Load(c, p, a)"), [precond_pos, precond_neg], [effect_add, effect_rem])

#  Unload
precond_pos = [expr("In(c, p)"), expr("At(p, a)"), expr("Cargo(c)"), expr("Plane(p)"), expr("Airport(a)")]
precond_neg = []
effect_add = [expr("At(c, a)")]
effect_rem = [expr("In(c, p)")]
unload = Action(expr("Unload(c, p, a)"), [precond_pos, precond_neg], [effect_add, effect_rem])

#  Fly
#  Used 'f' instead of 'from' because 'from' is a python keyword and expr uses eval() function
precond_pos = [expr("At(p, f)"), expr("Plane(p)"), expr("Airport(f)"), expr("Airport(to)")]
precond_neg = []
effect_add = [expr("At(p, to)")]
effect_rem = [expr("At(p, f)")]
fly = Action(expr("Fly(p, f, to)"), [precond_pos, precond_neg], [effect_add, effect_rem])

`precond_` is used to denote predicates which must be True **before** the action.
eg: 

`effect_` is used to denote predicates which must be True **after** the action.

`precon_pos` & `effect_add` are used to denote predicates which must be **True**.

`precon_neg` & `effect_rem` are used to denote predicates which must be **False**.


Eg: in  Action `load`

`precond_pos = [expr("At(c, a)"), expr("At(p, a)"), expr("Cargo(c)"), expr("Plane(p)"), expr("Airport(a)")]` is a  precondition which must be **True**   denoting predicate _Cargo c at Airport a & Plane p at Airport a_.

`precond_neg = []` is a precondition which must be **False**. 			

`effect_add = [expr("In(c, p)")]` is an effect which must be **True**  denoting the predicate _Cargo c in Plane p_.

`effect_rem = [expr("At(c, a)")]`is an effect which must be **False** denoting the predicate _Cargo c at Airport a_.

`load = Action(expr("Load(c, p, a)"), [precond_pos, precond_neg], [effect_add, effect_rem])`   thus define the action **`Load(c, p, a)** with the preconditions it must follow and the effects it leads to.

Finally, the function returns the defined problem using PDLL.

`return(PDLL(init, [load, unload, fly], goal_test)`

 which defines the problem as a whole with its initial state (`init`), the actions possible(` [load, unload, fly]`) and the goal(`goal_test`).

A solution to the air_cargo problem is as follows:

In [2]:
solution = [expr("Load(C1 , P1, SFO)"),
            expr("Fly(P1, SFO, JFK)"),
            expr("Unload(C1, P1, JFK)"),
            expr("Load(C2, P2, JFK)"),
            expr("Fly(P2, JFK, SFO)"),
            expr("Unload (C2, P2, SFO)")]

where `expr("Load(C1 , P1, SFO)")` means _load the Cargo C1 in Plane P1 at San Fransico_. In order for this action to take place the preconditions we specified while defining the action `Load` must be met.  The effects from this action are then carried forward i.e., they now are an existing state and it should be sought that these effects do **not** clash with preconditions of actions ahead, otherwise the action can't be completed.


We then execute the action on the state's kb.

In [3]:
a = air_cargo()

for action in solution:
    a.act(action)

where each action is acted upon leading to a state which is tested below to be the goal state or not.

In [4]:
a.goal_test()

True

Yes, the solution is correct. You may try any other solution to check if it achieves the goal or not.

# SPARE TIRE PROBLEM from Figure 10.2:

The problem follows the same structure of function as before, only the predicates are changed.


In [5]:
init = [expr('Tire(Flat)'),
            expr('Tire(Spare)'),
            expr('At(Flat, Axle)'),
            expr('At(Spare, Trunk)')]


required = [expr('At(Spare, Axle)'), expr('At(Flat, Ground)')]

In [6]:
# Actions

# Remove
precond_pos = [expr("At(obj, loc)")]
precond_neg = []
effect_add = [expr("At(obj, Ground)")]
effect_rem = [expr("At(obj, loc)")]
remove = Action(expr("Remove(obj, loc)"), [precond_pos, precond_neg], [effect_add, effect_rem])

# PutOn
precond_pos = [expr("Tire(t)"), expr("At(t, Ground)")]
precond_neg = [expr("At(Flat, Axle)")]
effect_add = [expr("At(t, Axle)")]
effect_rem = [expr("At(t, Ground)")]
put_on = Action(expr("PutOn(t, Axle)"), [precond_pos, precond_neg], [effect_add, effect_rem])

# LeaveOvernight
precond_pos = []
precond_neg = []
effect_add = []
effect_rem = [expr("At(Spare, Ground)"), expr("At(Spare, Axle)"), expr("At(Spare, Trunk)"),
              expr("At(Flat, Ground)"), expr("At(Flat, Axle)"), expr("At(Flat, Trunk)")]
leave_overnight = Action(expr("LeaveOvernight"), [precond_pos, precond_neg],
                         [effect_add, effect_rem])

solution to the spare tire problem from the book is as follows:

In [7]:
solution = [expr("Remove(Flat, Axle)"),
                expr("Remove(Spare, Trunk)"),
                expr("PutOn(Spare, Axle)")]

let's test it in the same way as before :

In [8]:
s = spare_tire()

for action in solution:
    s.act(action)

s.goal_test()

True

The solution achieves the goal.

# THE BLOCKS WORLD from Figure 10.3 :

 The above image below shows the transitions between different states which are possible for 3 blocks. Watch how the rightmost transition is what we follow in the book.

![pL plot](images/blocks_world.png)